In [12]:
# 데이터가 부족한 경우 사용
# NaN값을 제거하면 데이터가 5302개이므로 클롤링을 통해 NaN값을 채울려고 한다.
import pandas as pd
import numpy as np

- 데이터 분석(감독)

In [13]:
# title budget genres country main_charactor director vote_average vote_count
df = pd.read_csv('data/movie_data_original.csv',
                 names="title budget genres country main_charactor director vote_average vote_count".split(),
                 skiprows=1)

In [14]:
df.head(1)

,title,budget,genres,country,main_charactor,director,vote_average,vote_count
0,{616037: 'Thor: Love and Thunder'},250000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",{3894: 'Christian Bale'},{55934: 'Taika Waititi'},6.772,1752


In [35]:
# 감독이 없는 데이터 114개
NoDirectors = df[df['director'] == '-']
len(NoDirectors)

114

In [49]:
NoDirectors

,title,budget,genres,country,main_charactor,director,vote_average,vote_count
199,"{992187: ""Chip and Potato: Chip's Holiday""}",0,"[{'id': 16, 'name': 'Animation'}]","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",{76700: 'Briana Buckmaster'},-,5.3,6
207,{975117: 'Father There Is Only One 3'},0,[],[],{3062553: 'Bhavin Rabari'},-,6.8,3
221,{755433: 'The Monster at the End of This Story'},0,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...","[{'iso_3166_1': 'US', 'name': 'United States o...",{1213631: 'Leslie Carrara-Rudolph'},-,3.0,3
383,{575249: 'Dragon Ball Z: The Real 4-D at Super...,0,"[{'id': 28, 'name': 'Action'}, {'id': 16, 'nam...","[{'iso_3166_1': 'JP', 'name': 'Japan'}]",{90496: 'Masako Nozawa'},-,7.2,16
640,{464324: 'Depredador'},0,[],[],{55392: 'Martha Higareda'},-,6.0,2
...,...,...,...,...,...,...,...,...
9585,{934809: 'The Road to El Camino: Behind the Sc...,0,"[{'id': 99, 'name': 'Documentary'}]",[],{82945: 'Charles Baker'},-,6.5,4
9608,{624372: 'Ted Bundy: Mind of a Monster'},0,"[{'id': 80, 'name': 'Crime'}, {'id': 99, 'name...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",{1292615: 'Ted Bundy'},-,7.7,36
9628,"{993503: ""Bing's Animal Stories""}",0,[],[],{1138: 'Benoît Régent'},-,0.0,0
9929,{672291: 'Aladdin 2'},0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 10751, ...","[{'iso_3166_1': 'US', 'name': 'United States o...",{240724: 'Naomi Scott'},-,0.0,0


In [36]:
import ast

In [41]:
# 감독이 없는 리스트 중 타이틀을 string -> dict로 그중 title name만 가져옴
# Ndirector 에 리스트로 저장
Ndirector = []
for NoDirector in df.loc[NoDirectors.index].title:
    for s in ast.literal_eval(NoDirector).values():
        Ndirector.append(s)

In [45]:
Ndirector[0]

"Chip and Potato: Chip's Holiday"

- 클롤링(감독 데이터)

In [46]:
# wikipedia 검색 -> 첫번째 wiki사이트 클릭 -> 오른쪽 설명란에 direct by ["..."]
import requests

In [ ]:
search = "Chip and Potato: Chip's Holiday"
url = "https://en.wikipedia.org/w/index.php?search={search}&title=Special%3ASearch&go=Go&ns0=1"
